In [29]:
import requests
import pandas as pd
import numpy as np
from datetime import date
import yfinance as yf
import seaborn as sns
from sklearn.cluster import DBSCAN
from sklearn.datasets import make_blobs
from collections import defaultdict
import matplotlib.pyplot as plt

In [30]:
URL_BASE = 'https://api.stlouisfed.org/'
ENDPOINT = 'fred/series/observations'
API_KEY = "429c7bb6ba1c72479e276f06b345db1d"
URL = URL_BASE + ENDPOINT
start_date = "2021-03-01"
end_date = date.today().strftime("%Y-%m-%d")
INDICATOR = 'CORESTICKM159SFRBATL'

In [31]:
def fetch_fred_data(indicator, start_date=None, end_date=None, frequency='m'):
    """
    Pobiera dane z FRED API od zera za każdym razem
    
    Args:
        indicator (str): Kod wskaźnika FRED (np. 'CORESTICKM159SFRBATL')
        start_date (str): Data początkowa w formacie 'YYYY-MM-DD'
        end_date (str): Data końcowa w formacie 'YYYY-MM-DD' 
        frequency (str): Częstotliwość danych ('d', 'w', 'm', 'q', 'a')
    
    Returns:
        pd.DataFrame: Dane z FRED API
    """
    
    # Konfiguracja API
    URL_BASE = 'https://api.stlouisfed.org/'
    ENDPOINT = 'fred/series/observations'
    API_KEY = "429c7bb6ba1c72479e276f06b345db1d"  # Pamiętaj o zabezpieczeniu klucza!
    URL = URL_BASE + ENDPOINT
    
    # Parametry zapytania
    params = {
        'api_key': API_KEY,
        'series_id': indicator,
        'file_type': 'json',
        'frequency': frequency
    }
    
    # Dodanie dat jeśli podane
    if start_date:
        params['observation_start'] = start_date
    if end_date:
        params['observation_end'] = end_date
    
    print(f"Pobieranie danych dla wskaźnika: {indicator}")
    print(f"Okres: {start_date or 'początek'} - {end_date or 'koniec'}")
    
    try:
        # Wykonanie zapytania
        response = requests.get(URL, params=params)
        response.raise_for_status()  # Sprawdzenie błędów HTTP
        
        # Przetworzenie odpowiedzi
        data = response.json()
        
        if 'observations' not in data:
            print("Błąd: Brak danych w odpowiedzi API")
            return None
            
        # Normalizacja danych JSON
        df = pd.json_normalize(data["observations"])
        
        if df.empty:
            print("Ostrzeżenie: Pobrano pusty zbiór danych")
            return df
            
        # Usunięcie kolumn realtime (jeśli istnieją)
        columns_to_drop = ["realtime_start", "realtime_end"]
        df_cleaned = df.drop(columns=[col for col in columns_to_drop if col in df.columns], axis=1)
        
        # Konwersja dat i wartości
        if 'date' in df_cleaned.columns:
            df_cleaned['date'] = pd.to_datetime(df_cleaned['date'])
            
        if 'value' in df_cleaned.columns:
            # Zamiana '.' na NaN dla brakujących wartości
            df_cleaned['value'] = df_cleaned['value'].replace('.', np.nan)
            df_cleaned['value'] = pd.to_numeric(df_cleaned['value'], errors='coerce')
        
        print(f"Pobrano {len(df_cleaned)} obserwacji")
        return df_cleaned
        
    except requests.exceptions.RequestException as e:
        print(f"Błąd zapytania HTTP: {e}")
        return None
    except Exception as e:
        print(f"Błąd podczas przetwarzania danych: {e}")
        return None

# Funkcja do pobierania wielu wskaźników i łączenia w jeden DataFrame
def fetch_multiple_indicators(indicators, start_date=None, end_date=None, frequency='m'):
    """
    Pobiera dane dla wielu wskaźników i łączy je w jeden DataFrame
    
    Args:
        indicators (list): Lista kodów wskaźników
        start_date (str): Data początkowa
        end_date (str): Data końcowa
        frequency (str): Częstotliwość danych
        
    Returns:
        pd.DataFrame: Połączony DataFrame z wszystkimi wskaźnikami
    """
    all_dataframes = []
    
    for indicator in indicators:
        print(f"\n--- Pobieranie {indicator} ---")
        data = fetch_fred_data(indicator, start_date, end_date, frequency)
        if data is not None and not data.empty:
            # Zmiana nazwy kolumny 'value' na nazwę wskaźnika
            data_copy = data.copy()
            data_copy = data_copy.rename(columns={'value': indicator})
            # Zachowujemy tylko kolumny 'date' i wskaźnik
            data_copy = data_copy[['date', indicator]]
            all_dataframes.append(data_copy)
        else:
            print(f"Nie udało się pobrać danych dla {indicator}")
    
    if not all_dataframes:
        print("Nie pobrano żadnych danych")
        return pd.DataFrame()
    
    # Łączenie wszystkich DataFrame'ów po kolumnie 'date'
    combined_df = all_dataframes[0]
    for df in all_dataframes[1:]:
        combined_df = pd.merge(combined_df, df, on='date', how='outer')
    
    # Sortowanie po dacie
    combined_df = combined_df.sort_values('date').reset_index(drop=True)
    
    print(f"\nPołączono {len(all_dataframes)} wskaźników w jeden DataFrame")
    print(f"Kształt połączonego DataFrame: {combined_df.shape}")
    
    return combined_df

# Główna funkcja - uniwersalna dla pojedynczego wskaźnika lub wielu
def get_fred_data(indicators, start_date=None, end_date=None, frequency='m'):
    """
    Uniwersalna funkcja do pobierania danych FRED - zwraca zawsze pd.DataFrame
    
    Args:
        indicators (str lub list): Pojedynczy wskaźnik lub lista wskaźników
        start_date (str): Data początkowa w formacie 'YYYY-MM-DD'
        end_date (str): Data końcowa w formacie 'YYYY-MM-DD'
        frequency (str): Częstotliwość danych ('d', 'w', 'm', 'q', 'a')
    
    Returns:
        pd.DataFrame: DataFrame z danymi (pojedynczy wskaźnik lub połączone wskaźniki)
    """
    
    # Sprawdzenie czy indicators to string czy lista
    if isinstance(indicators, str):
        # Pojedynczy wskaźnik
        print(f"Pobieranie pojedynczego wskaźnika: {indicators}")
        result = fetch_fred_data(indicators, start_date, end_date, frequency)
        return result if result is not None else pd.DataFrame()
    
    elif isinstance(indicators, list):
        # Wiele wskaźników
        print(f"Pobieranie {len(indicators)} wskaźników: {', '.join(indicators)}")
        return fetch_multiple_indicators(indicators, start_date, end_date, frequency)
    
    else:
        print("Błąd: indicators musi być string lub lista")
        return pd.DataFrame()

## STIR regimes

WALCL - Assets: Total Assets: Total Assets
WREPODEL - Liabilities and Capital: Liabilities: Reverse Repurchase Agreements
WTREGEN - iabilities and Capital: Liabilities: Deposits with F.R. Banks, Other Than Reserve Balances: U.S. Treasury, General Account
WRESBAL - Liabilities and Capital: Other Factors Draining Reserve Balances: Reserve Balances with Federal Reserve Banks
RRPONTSYD - Overnight Reverse Repurchase Agreements: Treasury Securities Sold by the Federal Reserve in the Temporary Open Market Operations
EFFRVOL - Effective Federal Funds Volume 
SOFRVOL - Secured Overnight Financing Volume 
WREPOFOR - Liabilities and Capital: Liabilities: Reverse Repurchase Agreements: Foreign Official and International Accounts
WSEFINT1 - Memorandum Items: Custody Holdings: Securities in Custody for Foreign and International Accounts

In [52]:
df = get_fred_data(["WALCL", "WREPODEL",'WTREGEN','RRPONTSYD','EFFRVOL','SOFRVOL','WREPOFOR','WSEFINT1'],start_date=start_date, end_date=end_date, frequency='w')
df[['RRPONTSYD','EFFRVOL','SOFRVOL']]  =df[['RRPONTSYD','EFFRVOL','SOFRVOL']].shift(-1)
df = df.dropna()
df = df.drop(columns=['date'], axis=1)
df_log = (np.log(df) - np.log(df.shift(1).dropna())).dropna()
btc_df = yf.download('BTC-USD',start=start_date, interval='1wk')
btc_log = np.log(btc_df['Close'])
btc_log_chg = (btc_log - btc_log.shift(1)).dropna()

Pobieranie 8 wskaźników: WALCL, WREPODEL, WTREGEN, RRPONTSYD, EFFRVOL, SOFRVOL, WREPOFOR, WSEFINT1

--- Pobieranie WALCL ---
Pobieranie danych dla wskaźnika: WALCL
Okres: 2021-03-01 - 2025-07-13
Pobrano 228 obserwacji

--- Pobieranie WREPODEL ---
Pobieranie danych dla wskaźnika: WREPODEL
Okres: 2021-03-01 - 2025-07-13
Pobrano 228 obserwacji

--- Pobieranie WTREGEN ---
Pobieranie danych dla wskaźnika: WTREGEN
Okres: 2021-03-01 - 2025-07-13
Pobrano 228 obserwacji

--- Pobieranie RRPONTSYD ---
Pobieranie danych dla wskaźnika: RRPONTSYD
Okres: 2021-03-01 - 2025-07-13
Pobrano 228 obserwacji

--- Pobieranie EFFRVOL ---
Pobieranie danych dla wskaźnika: EFFRVOL
Okres: 2021-03-01 - 2025-07-13
Pobrano 228 obserwacji

--- Pobieranie SOFRVOL ---
Pobieranie danych dla wskaźnika: SOFRVOL
Okres: 2021-03-01 - 2025-07-13
Pobrano 228 obserwacji

--- Pobieranie WREPOFOR ---
Pobieranie danych dla wskaźnika: WREPOFOR
Okres: 2021-03-01 - 2025-07-13
Pobrano 228 obserwacji

--- Pobieranie WSEFINT1 ---
Pobiera

/var/folders/m9/j3jd25ls7cbbv4dpcfjkkhw40000gn/T/ipykernel_974/3144655517.py:6: FutureWarning:

YF.download() has changed argument auto_adjust default to True

[*********************100%***********************]  1 of 1 completed


In [53]:
#sns.pairplot(df_log)

In [169]:
dbscan = DBSCAN(eps=0.1, min_samples=5)
model = dbscan.fit_predict(btc_log_chg)
btc_log_chg['cluster']= model


In [170]:
import plotly.express as px


px.scatter(y=btc_df['Close']['BTC-USD'].shift(1).dropna(), color=btc_log_chg['cluster'])

In [171]:
from sklearn.metrics import silhouette_score as ss

ss(pd.DataFrame(btc_log_chg['BTC-USD']), btc_log_chg['cluster'])

np.float64(-0.04614230710047578)

In [172]:
pd.DataFrame(btc_log_chg['BTC-USD'])

,BTC-USD
Date,
2021-03-08,0.146778
2021-03-15,-0.030456
2021-03-22,-0.027720
2021-03-29,0.048965
2021-04-05,0.024318
...,...
2025-06-09,-0.002287
2025-06-16,-0.044211
2025-06-23,0.070702


## Take all daily tickers from fred and check best ss test

## Macro regimes

In [ ]:
df = get_fred_data(['VIXCLS','DCOILWTICO',],start_date=start_date, end_date=end_date, frequency='w')
